In [ ]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import os
import pykonal
import seispy

GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']

In [ ]:
vm = seispy.velocity.VelocityModel(
    os.path.join(
        GOOGLE_DRIVE,
        'malcolm.white@usc.edu',
        'data',
        'velocity',
        'White_et_al_2019a',
        'White_et_al_2019a.regular.npz'
    ),
    fmt='npz'
)

In [ ]:
solver = pykonal.EikonalSolver()
solver.mode = 'hybrid-spherical'
solver.vgrid.min_coords     = 1000, np.pi/2, 0
solver.vgrid.node_intervals = 200, np.pi/10, np.pi/40
solver.vgrid.npts           = 20, 1, 21
solver.vv                   = np.ones(solver.vgrid.npts)
# for ir in range(solver.vgrid.npts[0]):
#     solver.vv[ir] *= (ir + 1)
solver.pgrid.min_coords     = solver.vgrid.min_coords
solver.pgrid.node_intervals = solver.vgrid.node_intervals
solver.pgrid.npts           = solver.vgrid.npts 
solver.src_loc              = 3525, np.pi/2, np.pi/6
# solver._solve_near_field()
solver._solve_hybrid()

In [ ]:
# vmin, vmax = near_field.uu.min(), near_field.uu.max()
vmin, vmax = solver.uu.min(), solver.uu.max()

vfar    = solver.vgrid[...]
xx_far  = vfar[:,:,:,0] * np.sin(vfar[:,:,:,1]) * np.cos(vfar[:,:,:,2])
yy_far  = vfar[:,:,:,0] * np.sin(vfar[:,:,:,1]) * np.sin(vfar[:,:,:,2])
zz_far  = vfar[:,:,:,0] * np.cos(vfar[:,:,:,1])

vnear = solver.near_field.vgrid
xx_near = vnear[:,:,:,0] * np.sin(vnear[:,:,:,1]) * np.cos(vnear[:,:,:,2]) + solver.src_xyz[0]
yy_near = vnear[:,:,:,0] * np.sin(vnear[:,:,:,1]) * np.sin(vnear[:,:,:,2]) + solver.src_xyz[1]
zz_near = vnear[:,:,:,0] * np.cos(vnear[:,:,:,1]) + solver.src_xyz[2]

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect=1)
# ax.scatter(
#     xx_far, 
#     yy_far,
#     s=20,
#     c='w',
#     edgecolor='k'
# )
ax.scatter(
    xx_far, 
    yy_far,
    s=20,
    c=solver.uu, 
    cmap=plt.get_cmap('rainbow'),
    edgecolor='k',
    vmin=vmin,
    vmax=vmax
)
# ax.scatter(
#     xx_near, 
#     yy_near,
#     s=5,
#     c=near_field.uu,
#     cmap=plt.get_cmap('jet_r'),
#     vmin=vmin,
#     vmax=vmax
# )
ax.scatter(solver.src_xyz[0], solver.src_xyz[1], marker='*', c='r', edgecolor='k', s=200)

In [ ]:
solver.norm.shape

In [ ]:
solver = pykonal.EikonalSolver()
solver.mode = 'hybrid-cartesian'
solver.vgrid.min_coords     = 0, 0, 0
solver.vgrid.node_intervals = 1, 1, 1
solver.vgrid.npts           = 100, 100, 50
solver.vv                   = np.ones(solver.vgrid.npts)
solver.src_loc              = 50, 50, 25
solver.solve()
src_solver                  = solver.src_solver

uu0 = np.sqrt(np.sum(np.square(solver.pgrid[...] - solver.src_loc), axis=-1))
# np.any(np.isnan(solver.uu)), np.any(np.isinf(solver.uu))

In [ ]:
vmin, vmax = -0.2, 0.2
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect=1)

xx = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.cos(src_solver.pgrid[..., 2]) + solver.src_loc[0]
yy = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.sin(src_solver.pgrid[..., 2]) + solver.src_loc[1]
zz = src_solver.pgrid[..., 0] * np.cos(src_solver.pgrid[..., 1]) + solver.src_loc[2]
uu0 = np.sqrt(np.square(xx-solver.src_loc[0]) + np.square(yy-solver.src_loc[1]) + np.square(zz-solver.src_loc[2]))
# qmesh = ax.pcolormesh(xx[:,0,:], yy[:,0,:], 
#     (src_solver.uu[:,0,:]-uu0[:,0,:])/uu0[:,0,:],
#     vmin=vmin,
#     vmax=vmax,
#     cmap=plt.get_cmap('jet_r')
# )

xx = solver.pgrid[..., 0]
yy = solver.pgrid[..., 1]
zz = solver.pgrid[..., 2]

uu0 = np.sqrt(np.square(xx-solver.src_loc[0]) + np.square(yy-solver.src_loc[1]) + np.square(zz-solver.src_loc[2]))
qmesh = ax.pcolormesh(xx[:,:,0], yy[:,:,0],
    ((solver.uu[:,:,0]-uu0[:,:,0])/uu0[:,:,0])*100,
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r')
)
ax.scatter(solver.src_loc[0], solver.src_loc[1], marker='x', color='k')

cbar = fig.colorbar(qmesh, ax=ax)

In [ ]:
vmin, vmax = 0, np.max(np.abs(solver.uu))

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, aspect=1)

xx = solver.pgrid[..., 0]
yy = solver.pgrid[..., 1]
zz = solver.pgrid[..., 2]
qmesh = ax.pcolormesh(
    xx[:,:,0]-solver.pgrid.node_intervals[0]/2, 
    yy[:,:,0]-solver.pgrid.node_intervals[1]/2,
    solver.uu[:,:,0],
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r'),
)

xx = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.cos(src_solver.pgrid[..., 2]) + solver.src_loc[0]
yy = src_solver.pgrid[..., 0] * np.sin(src_solver.pgrid[..., 1]) * np.sin(src_solver.pgrid[..., 2]) + solver.src_loc[1]
zz = src_solver.pgrid[..., 0] * np.cos(src_solver.pgrid[..., 1]) + solver.src_loc[2]
qmesh = ax.scatter(xx[:,0,:], yy[:,0,:], 
    c=src_solver.uu[:,0,:],
    vmin=vmin,
    vmax=vmax,
    cmap=plt.get_cmap('jet_r'),
    edgecolor='k'
)

cbar = fig.colorbar(qmesh, ax=ax)

In [ ]:
img = itk.GetImageFromArray(solver.uu)

In [ ]:
image = itk.imread(file_name)
view(img)

In [ ]:
solver.uu.max()